In [ ]:
# Calculate the total revenue of each product at each price. The solution must be applicable to similar datasets with large number of records
# Datasets: prices.csv(records of product's history price changes in Sep 2018), sales.csv(records of product's sales in Sep 2018)

In [2]:
import pandas as pd
import numpy as np
sales = pd.read_csv("sales.csv")
prices = pd.read_csv("prices.csv")
full_table = sales.merge(prices, on ='product_id', how = 'inner')
print(full_table)

     product_id     ordered_at  quantity_ordered  old_price  new_price  \
0       3998909  9/18/18 17:51                 1      15500      16500   
1       3998909  9/18/18 17:51                 1      17000      15500   
2       3998909  9/18/18 17:51                 1      19000      17000   
3       3998909  9/18/18 12:52                 1      15500      16500   
4       3998909  9/18/18 12:52                 1      17000      15500   
..          ...            ...               ...        ...        ...   
949     4085861  9/13/18 23:29                 1      67000      62500   
950          64  9/11/18 19:32                 1     270000     239000   
951          64  9/14/18 20:10                 1     270000     239000   
952          64  9/12/18 16:00                 1     270000     239000   
953          64  9/12/18 13:23                 1     270000     239000   

        updated_at  
0     9/16/18 5:09  
1     9/13/18 6:43  
2    9/10/18 16:35  
3     9/16/18 5:09  
4     

In [3]:
full_table['ordered_at'] = pd.to_datetime(full_table['ordered_at'])
full_table['updated_at'] = pd.to_datetime(full_table['updated_at'])


def right_order_time (id, order):
    order_time = full_table['updated_at'][full_table['product_id']==id]
    if order < order_time.min():
        real_order_time = order_time.min()
    else:
        real_order_time = order_time[order>order_time].max()
    return real_order_time

def right_price(id, order):
    right_time = right_order_time(id, order)
    if order >= right_time:
        real_price = full_table['new_price'][(full_table['product_id']==id) & (full_table['updated_at'] == right_time)].values[0]
    else:
        real_price = full_table['old_price'][(full_table['product_id']==id) & (full_table['updated_at'] == right_time)].values[0]
    return real_price


final_price = []
for i in np.arange(len(full_table)):
    finalprice = right_price(full_table.iloc[i]['product_id'], full_table.iloc[i]['ordered_at']) 
    final_price.append(finalprice)

full_table['price'] = final_price
full_table['revenue'] = full_table['price'] * full_table['quantity_ordered']
    
print(full_table['revenue'])
full_table.to_csv('revenue.csv')
print(full_table.groupby('product_id').agg({'revenue':sum}))

0       16500
1       16500
2       16500
3       16500
4       16500
        ...  
949     62500
950    239000
951    239000
952    239000
953    239000
Name: revenue, Length: 954, dtype: int64
             revenue
product_id          
64            956000
3954203      3510000
3998909       841500
4085861     49485000
